In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
if '/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/' not in sys.path:
  sys.path.append('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Aug  8 13:27:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

DICOM dataset to jpeg

In [ ]:
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 8.5 MB/s 


In [ ]:
import os
import pydicom as dicom
import PIL # optional
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
i=0
for path, subdirs, files in os.walk('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/Prostate_dcm/manifest-A3Y4AE4o5818678569166032044/'):
    for name in files:
      if name.endswith('.dcm'):
        image_path = os.path.join(path, name)
        ds = dicom.dcmread(image_path)
        if len(ds.pixel_array.shape)==2: # checks if the pixel array in the DICOM file has 2 dimensions using the shape 
                                         # attribute of the pixel_array property of the DICOM object ds
          plt.imshow(ds.pixel_array, cmap='gray')
          plt.axis('off')
          my_dpi=120
          save_path = image_path.replace('.dcm', '.png')
          plt.savefig(save_path, bbox_inches='tight',  # bbox_inches='tight' trims any whitespace around the edges of the plot
                                 pad_inches = 0)       # pad_inches=0 removes padding btw plot and the edge of the saved .png
          plt.figure(figsize=(384/my_dpi, 384/my_dpi), dpi=my_dpi)
          plt.show()
          plt.close()
          image = Image.open(save_path)
          resized_image = image.resize((384,384))
          resized_image = resized_image.save(save_path)
          if i%20==0:
            print('Image number:', i, ' ', image_path)
          i+=1
        else:       
          continue # If the pixel array has more than 2 dimensions, the code skips to the next file
      else:         
        continue   # If the file does not end with '.dcm', the code also skips to the next file

This code loads DICOM files from a directory, converts them to PNG format, resizes them to 384x384 pixels, and saves them back to the same directory. It also displays the original and resized images using Matplotlib. this code is useful for converting DICOM files to PNG format and resizing them toa specific size,

In [ ]:
from PIL import Image

for path, subdirs, files in os.walk('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/Prostate_dcm/TestData/'):
    for name in files:
      if name.endswith('.raw'):
        image_path = os.path.join(path, name)
        rawData = open(image_path, 'rb').read() # open a file image_path in binary mode ('rb')
        imgSize = (384,384) # the image size
        img = Image.frombytes('L',              # creates a PIL image object from the raw image
                              imgSize, rawData) # 'L' specifies the mode if the image (grayscale)
        save_path = image_path.replace('.raw', '.png')
        img.save(save_path) # can give any format you like .png
      else:
        continue


This code reads raw image files, converts them to PNG format, and saves them in the same directory. Binary mode is used here because the file being read is not a text file, but rather an image file. Reading a binary file in text mode can cause issues, such as the conversion of line endings or other special characters. Once the contents of the file are stored in rawData, they can be processed or manipulated as needed. For example, they can be passed to a library or function that expects image data in bytes format.

In [ ]:
import numpy as np
import os    # Traverse folders 
import nibabel as nib #nii Format 1 This bag will be used in general 
import imageio   # Convert to an image 
 
def nii_to_image(niifile):
 filenames = os.listdir(filepath) # Read nii Folder
 slice_trans = []
 
 for f in filenames:
  # Start reading nii Documents 
  img_path = os.path.join(filepath, f)
  img = nib.load(img_path)    # Read nii
  img_fdata = img.get_fdata() # returns a numpy array of the image data from a NIfTI file
  fname = f.replace('.nii','')   # Remove nii Suffix name of 
  img_f_path = os.path.join(imgfile, fname)
  # Create nii The folder of the corresponding image 
  if not os.path.exists(img_f_path):
   os.mkdir(img_f_path)    # create New Folder 
 
  # Start converting to an image 
  (x,y,z) = img.shape
  for i in range(z):      #z Is a sequence of images 
   silce = img_fdata[i, :, :]   # You can choose which direction of slice 
   imageio.imwrite(os.path.join(img_f_path,'{}.png'.format(i)), silce) # The function takes two arguments: 
                                                                       # the path to the file where the image should be saved, 
                                                                       # and the image data as a numpy array
filepath = '/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/Prostate_dcm/Processed_data_nii/BIDMC'
imgfile = '/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/Prostate_dcm/Processed_data_nii/BIDMC'
nii_to_image(filepath)

This code is a Python function that converts MRI images in NIfTI format (.nii files) to PNG images and saves them in a corresponding directory for each NIfTI file. the function takes a single argument `niifile`, which is the path to the folder containing the NIfTI files. It then reads all the filenames in this folder using the `os.listdir()` method and stores them in the `filenames` variable. For each file in `filenames`, the function loads the NIfTI file using the `nib.load()` method from the `nibabel` library, which returns an `nibabel.nifti1.Nifti1Image` object. The function then extracts the image data as a numpy array using the `get_fdata()` method and stores it in the `img_fdata` variable.

The function then creates a new directory with the same name as the NIfTI file (with the `.nii` extension removed) in the specified `imgfile` directory using the `os.mkdir()` method. This directory will be used to store the PNG images that correspond to the slices of the NIfTI image.

The function then loops through each slice of the NIfTI image along the `z` axis (which corresponds to the sequence of images), and saves each slice as a PNG image in the corresponding directory using the `imageio.imwrite()` method.

NIfTI (Neuroimaging Informatics Technology Initiative) is a standard file format for storing medical imaging data, particularly for brain imaging studies such as magnetic resonance imaging (MRI) and positron emission tomography (PET).

A NIfTI file contains both the image data and metadata such as image dimensions, voxel size, and orientation information. The format is designed to be simple, flexible, and extensible, making it widely used in neuroimaging research.

A NIfTI image is typically a 3D or 4D array of image data, where the dimensions correspond to the x, y, z, and time (if present) of the image. Each element in the array represents the signal intensity at a specific spatial location (voxel) in the image, which can be used to visualize and analyze the structure and function of the brain.

The NIfTI format also includes a header that contains information about the image acquisition parameters, such as the imaging modality, the orientation of the image, and the voxel size. This header information is essential for correctly interpreting the image data and ensuring that the image is analyzed correctly.

Metadata is data that describes other data. In the context of medical imaging, metadata provides information about the image data, such as the imaging modality, acquisition parameters, and patient demographics.

Metadata is typically stored in a header file that accompanies the image data. In the case of NIfTI format, the metadata is stored in the header section of the file, which is separate from the image data.

In [ ]:
for path, subdirs, files in os.walk('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/Prostate_dcm/manifest-A3Y4AE4o5818678569166032044/'):
    for name in files:
      if name.endswith('.dcm'):
        image_path = os.path.join(path, name)
        os.remove(image_path) # deletes the file from the file system
        print('File removed: ', image_path)
      else:
        continue

This code is useful for removing unwanted DICOM files from a directory tree. In this case, the code is specifically removing DICOM files from the directory tree specified by the root directory.

In [ ]:
import shutil
for path, subdirs, files in os.walk('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/Prostate_dcm/manifest-A3Y4AE4o5818678569166032044/'):
    for name in subdirs:
      if 'reftra' in name:
        image_path = os.path.join(path, name)
        #os.remove(image_path)
        shutil.rmtree(image_path) # deletes the directory and all its contents recursively.
        print('File removed: ', image_path)
      else:
        continue

This code is similar to the previous code, but instead of removing individual files, it removes entire directories using the shutil module.